In [95]:
# Paso 1.
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.
import pandas as pd
import numpy as np

dataset_test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col=False,
    compression="zip",
)
dataset_train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col=False,
    compression="zip",
)

dataset_test['Age']=2021-dataset_test['Year']
dataset_train['Age']=2021-dataset_train['Year']

dataset_test.pop('Year')
dataset_test.pop('Car_Name')
dataset_train.pop('Year')
dataset_train.pop('Car_Name')

dataset_test.head()


,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,4.75,9.54,43000,Diesel,Dealer,Manual,0,8
1,7.25,9.85,6900,Petrol,Dealer,Manual,0,4
2,2.85,4.15,5200,Petrol,Dealer,Manual,0,10
3,6.75,8.12,18796,Petrol,Dealer,Manual,0,6
4,6.50,8.61,33429,Diesel,Dealer,Manual,0,6


In [96]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.

y_train=dataset_train.copy()
y_train=y_train.pop("Present_Price")

x_train = dataset_train.copy()
x_train.pop("Present_Price")

y_test=dataset_test.copy()
y_test=y_test.pop("Present_Price")

x_test = dataset_test.copy()
x_test.pop("Present_Price")

0      9.54
1      9.85
2      4.15
3      8.12
4      8.61
      ...  
85    13.60
86    13.60
87     7.60
88     8.40
89    11.60
Name: Present_Price, Length: 90, dtype: float64

In [97]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression

categorical_columns = ["Fuel_Type", "Selling_type", "Transmission"]
numeric_columns = [col for col in x_train.columns if col not in categorical_columns]


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns), 
        ('num', MinMaxScaler(), numeric_columns)          
    ],
    remainder='passthrough'  
)

k_best_selector = SelectKBest(score_func=f_classif, k=10)

linear_model = LinearRegression()

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),   
    ('feature_selector', k_best_selector),  
    ('regre', linear_model)    
])

pipeline.fit(x_train, y_train)

c:\Users\usuario\Downloads\2024-2-LAB-05-prediccion-de-precios-usando-linreg-dzaptaca\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['Fuel_Type', 'Selling_type',
                                                   'Transmission']),
                                                 ('num', MinMaxScaler(),
                                                  ['Selling_Price',
                                                   'Driven_kms', 'Owner',
                                                   'Age'])])),
                ('feature_selector', SelectKBest()),
                ('regre', LinearRegression())])

In [ ]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.

from sklearn.model_selection import GridSearchCV

param_grid = {
    'feature_selector__k':[1,2,3,4,5,6,7,8,9,10,11],
}


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',  
    cv=10,                                      
    n_jobs=-1,                                  
    verbose=1                                    
)


grid_search.fit(x_train, y_train)


print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)
print(f"Mejor precisión balanceada (validación cruzada): {grid_search.best_score_:.4f}")


best_pipeline = grid_search.best_estimator_

Fitting 10 folds for each of 8 candidates, totalling 80 fits

Mejores parámetros encontrados:
{'feature_selector__k': 7}
Mejor precisión balanceada (validación cruzada): -1.8915


c:\Users\usuario\Downloads\2024-2-LAB-05-prediccion-de-precios-usando-linreg-dzaptaca\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


In [99]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

import os
import pickle
import gzip

dir_path = '../files/models'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(grid_search, f)
else:
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(grid_search, f)

In [100]:
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

import os
import json
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error


def calculate_metrics(y_true, y_pred, dataset_name):
    metrics = {
        'type': 'metrics',
        'dataset': dataset_name,
        'r2': r2_score(y_true, y_pred),
        'mse': mean_squared_error(y_true, y_pred),
        'mad': median_absolute_error(y_true, y_pred),
    }
    return metrics


y_train_pred = best_pipeline.predict(x_train)
y_test_pred = best_pipeline.predict(x_test)


train_metrics = calculate_metrics(y_train, y_train_pred, 'train')
test_metrics = calculate_metrics(y_test, y_test_pred, 'test')


output_dir = "../files/output"
output_path = os.path.join(output_dir, "metrics.json")


os.makedirs(output_dir, exist_ok=True)


with open(output_path, 'w') as f:
    json.dump(train_metrics, f)
    f.write("\n")
    json.dump(test_metrics, f)


print(f"\nMétricas guardadas exitosamente en '{output_path}'")


Métricas guardadas exitosamente en '../files/output\metrics.json'
